In [82]:
import pandas as pd
import json
from pprint import pprint

In [83]:
unigramData = pd.read_csv('unigramLetterData.csv')

In [84]:
bigramData = None

with open('bigramFiles/bigrams.json') as json_data:
    d = json.load(json_data)
    bigramData = pd.DataFrame.from_dict(d)
bigramData.head()    

,0,1
0,th,100272945963
1,he,86697336727
2,in,68595215308
3,er,57754162106
4,an,55974567611


In [85]:
bigramData.columns = ['Bigram', 'Percentage']
total_freq = bigramData['Percentage'].sum()
bigramData['Percentage'] = (bigramData['Percentage']/float(total_freq))*100
bigramData.head()

,Bigram,Percentage
0,th,3.556203
1,he,3.074741
2,in,2.432745
3,er,2.048265
4,an,1.985151


In [86]:
unigramData.head()

,Letter,Percentage
0,E,12.02
1,T,9.10
2,A,8.12
3,O,7.68
4,I,7.31


In [87]:
file = open('node dictionary/filteredWords/threeLetterWords.txt', 'r') 
threeLetterWords = file.readlines()
threeLetterWords = [s.rstrip() for s in threeLetterWords]
file = open('node dictionary/filteredWords/fourLetterWords.txt', 'r') 
fourLetterWords = file.readlines()
fourLetterWords = [s.rstrip() for s in fourLetterWords]
file = open('node dictionary/filteredWords/fiveLetterWords.txt', 'r') 
fiveLetterWords = file.readlines()
fiveLetterWords = [s.rstrip() for s in fiveLetterWords]
file = open('node dictionary/filteredWords/sixLetterWords.txt', 'r') 
sixLetterWords = file.readlines()
sixLetterWords = [s.rstrip() for s in sixLetterWords]

In [88]:
word = raw_input()
guess_word_list = []
last_guessed_letter = '*'
penalty = 0
guess_word = "*" * len(word)

humble


In [89]:
def replaceLetter(word, guess_word, letter,):
    new_word = ""
    for i in range(len(word)):
        if word[i] == letter:
            new_word += letter
        elif guess_word[i].isalpha() == True:
            new_word += word[i]
        else:
            new_word += "*"
    return new_word       

In [90]:
def guessLetter( word, guess_word, letter):
    if letter in word:
        guess_word = replaceLetter(word,guess_word, letter)
    return guess_word

In [91]:
def guess_first_letter(word, guess_word, penalty):
    for i in range(26):
        new_word = guessLetter(word, guess_word, unigramData['Letter'].iloc[i].lower())
        if new_word != guess_word:
            guess_word_list.append(unigramData['Letter'].iloc[i].lower())
            last_guessed_letter = unigramData['Letter'].iloc[i].lower()
            return (new_word, penalty, last_guessed_letter)
        else:
            penalty += 1
        

In [92]:
def getString( string, letter):
    if string[0] == letter:
        return 'L'
    elif string[1] == letter:
        return 'R'
    else:
        return 'X'

In [93]:
def maxProbBigram(letter):
    bigram_dict = bigramData['Bigram'].apply(lambda x: getString(x, letter))
    left_letter_data = bigramData[(bigram_dict == 'L')]
    right_letter_data = bigramData[(bigram_dict == 'R')]
    return (left_letter_data, right_letter_data)

In [94]:
def guessLetterHelper(word, guess_word, letter,penalty):
    (left_letter_data, right_letter_data) = maxProbBigram(letter)
    left_counter = 0
    right_counter = 0
    while(1):
        left_letter = left_letter_data.iloc[left_counter, :]
        right_letter = right_letter_data.iloc[right_counter, :]
        if left_letter['Percentage'] > right_letter['Percentage']:
            if left_letter['Bigram'][1] not in guess_word_list:
                new_word = guessLetter(word, guess_word, left_letter['Bigram'][1])
                guess_word_list.append(left_letter['Bigram'][1])
                if new_word != guess_word:
                    last_guessed_letter = left_letter['Bigram'][1]
                    return (new_word, penalty, last_guessed_letter)
                else:
                    left_counter += 1
                    penalty += 1
            else:
                left_counter += 1
        else:
            if right_letter['Bigram'][1] not in guess_word_list:
                new_word = guessLetter(word, guess_word, right_letter['Bigram'][0])
                guess_word_list.append(right_letter['Bigram'][0])
                if new_word != guess_word:
                    last_guessed_letter = right_letter['Bigram'][0]
                    return (new_word, penalty, last_guessed_letter)
                else:
                    right_counter += 1
                    penalty += 1
            else:
                right_counter += 1
                

In [95]:
(guess_word, penalty, last_guessed_letter) = guess_first_letter(word, guess_word, penalty)
print guess_word
print guess_word_list
print penalty
print last_guessed_letter
while( word != guess_word):
    (guess_word, penalty, last_guessed_letter) = guessLetterHelper(word, guess_word,last_guessed_letter, penalty)
    print guess_word
    print guess_word_list
    print penalty
    print last_guessed_letter

*****e
['e']
0
e
****le
['e', u'r', u'n', u's', u'd', u'a', u'l']
5
l
*u**le
['e', u'r', u'n', u's', u'd', u'a', u'l', u'i', u'y', u'o', u'u']
8
u
*um*le
['e', u'r', u'n', u's', u'd', u'a', u'l', u'i', u'y', u'o', u'u', u't', u'c', u'm']
10
m
*umble
['e', u'r', u'n', u's', u'd', u'a', u'l', u'i', u'y', u'o', u'u', u't', u'c', u'm', u'p', u'b']
11
b
humble
['e', u'r', u'n', u's', u'd', u'a', u'l', u'i', u'y', u'o', u'u', u't', u'c', u'm', u'p', u'b', u'j', u'v', u'h']
13
h
